In [ ]:
using CSV, Plots, Random, LinearAlgebra, Statistics, DataFrames

In [ ]:
num_of_model = 3
split = [43, 43+44, 43+44*2, 43+44*3, 43+44*4, 43+44*5]  # index for 5-fold

function select_best_model2(stock)  # using k-fold cross validation
    err = zeros(num_of_model,5)
    w = zeros(21)
    
    for i in 1:5
        w_set = []
        # setting train and validation set
        split_cur = split[i]
        split2 = Int(round(split_cur*4/5))
        
        stock_train = stock[7][1:split2]
        stock_val = stock[7][split2 + 1:split_cur]
        
        sent_train = stock[1][1:split2]
        sent_val = stock[1][split2 + 1:split_cur]
        
        trav_train = stock[6][1:split2] #travelling
        trav_val = stock[6][split2 + 1:split_cur]
        
        death_train = stock[3][1:split2]
        death_val = stock[3][split2 + 1:split_cur]
        
        vac_train = stock[2][1:split2]
        vac_val = stock[2][split2 + 1:split_cur]
        
        shut_train = stock[4][1:split2]
        shut_val = stock[4][split2 + 1:split_cur]
    
        # Naive Model, w[1:6]
        Xtrain = [sent_train trav_train death_train vac_train shut_train ones(length(sent_train))]
        Xval = [sent_val trav_val death_val vac_val shut_val ones(length(sent_val))]
        Ytrain = stock_train
        Yval = stock_val
    
        w_cur = Xtrain\Ytrain
        w_set = vcat(w_set, w_cur)
        pred = Xval*w_cur
        err[1,i] = abs_err(Yval,pred)
    
        # AR1 Model, w[7:13] 1 to end - 1
        Xtrain = [sent_train[2:end] stock_train[1:end-1] trav_train[2:end] death_train[2:end] vac_train[2:end] shut_train[2:end] ones(length(sent_train)-1)]
        Xval = [sent_val[2:end] stock_val[1:end-1] trav_val[2:end] death_val[2:end] vac_val[2:end] shut_val[2:end] ones(length(sent_val)-1)]
        Ytrain = stock_train[2:end]
        Yval = stock_val[2:end]
    
        w_cur = Xtrain\Ytrain
        w_set = vcat(w_set, w_cur)
        pred = Xval*w_cur
        err[2,i] = abs_err(Yval,pred)
    
        # AR2 Model, w[14:21]
        Xtrain = [sent_train[3:end] stock_train[2:end-1] stock_train[1:end-2] trav_train[3:end] death_train[3:end] vac_train[3:end] shut_train[3:end] ones(length(sent_train)-2)]
        Xval = [sent_val[3:end] stock_val[2:end-1] stock_val[1:end-2] trav_val[3:end] death_val[3:end] vac_val[3:end] shut_val[3:end] ones(length(sent_val)-2)]
        Ytrain = stock_train[3:end]
        Yval = stock_val[3:end]
    
        w_cur = Xtrain\Ytrain
        w_set = vcat(w_set, w_cur)
        pred = Xval*w_cur
        err[3,i] = abs_err(Yval,pred)
        w = hcat(w,w_set)
    end
    
    avg_err = mean(err, dims=2)   # avg_abs_err
    final_model = findmin(avg_err)[2][1]   # select model with min abs error
    print("Low train error")
    test_err = zeros(5)
    model = ["Naive Model", "AR1 Model", "AR2 Model"]
    
    for i in 1:5
        stock_test = stock[7][split[2]-44:split[2]]
        sent_test = stock[1][split[2]-44:split[2]]
        trav_test = stock[6][split[2]-44:split[2]]
        death_test = stock[3][split[2]-44:split[2]]
        vac_test = stock[2][split[2]-44:split[2]]
        shut_test = stock[4][split[2]-44:split[2]]
    
        if final_model == 1   # Naive Model
            Xtest = [sent_test trav_test death_test vac_test shut_test ones(length(sent_test))]
            Ytest = stock_test
    
            w_cur = w[1:6, i+1]
            pred = Xtest*w_cur
            test_err[i] = abs_err(Ytest,pred)
        
        elseif final_model == 2   # AR1 Model
            Xtest = [sent_test[2:end] stock_test[1:end-1] trav_test[2:end] death_test[2:end] vac_test[2:end] shut_test[2:end] ones(length(stock_test)-1)]
            Ytest = stock_test[2:end]
        
            w_cur = w[7:13, i+1]
            pred = Xtest*w_cur
            test_err[i] = abs_err(Ytest,pred)
        
        elseif final_model == 3   # AR2 Model
            Xtest = [sent_test[3:end] stock_test[2:end-1] stock_test[1:end-2] trav_test[3:end] death_test[3:end] vac_test[3:end] shut_test[3:end] ones(length(stock_test)-2)]
            Ytest = stock_test[3:end]
        
            w_cur = w[13:21, i+1]
            pred = Xtest*w_cur
            test_err[i] = abs_err(Ytest,pred)
        
        else
            print("error")
        end
    end
    string("selected model: ", model[final_model], ", estimated error is: ", mean(test_err))
end